In [55]:
# import geemap
import geemap.eefolium as geemap
import ee
import numpy as np
import pandas as pd

from egis.utils import (
    vis_params_dict, default_vis_params, get_known_roi, extract_single_img, 
    extract_img_collection_metadata, extract_img_metadata, extract_img_collection_properties,
    coords_to_polygon, collection_len
)

from IPython.core.display import display, HTML

In [2]:
ee.Initialize()

In [3]:
Map = geemap.Map()

In [4]:
# Daily mean 2m air temperature
era5_2mt = ee.ImageCollection('ECMWF/ERA5/DAILY') \
                   .select('mean_2m_air_temperature') \
                   .filter(ee.Filter.date('2019-07-01', '2019-07-31'))

In [ ]:
coll_info = era5_2mt.getInfo()

In [59]:
coll_info.keys()

dict_keys(['type', 'bands', 'id', 'version', 'properties', 'features'])

In [60]:
coll_info['properties']

{'system:visualization_0_min': '250.0',
 'type_name': 'ImageCollection',
 'max_mirrored_version': 1594797432309553,
 'keywords': ['climate',
  'copernicus',
  'dewpoint',
  'ecmwf',
  'era5',
  'precipitation',
  'pressure',
  'reanalysis',
  'surface',
  'temperature',
  'wind'],
 'system:visualization_1_palette': '#FFFFFF,#00FFFF,#0080FF,#DA00FF,#FFA400,#FF0000',
 'visualization_1_bands': 'total_precipitation',
 'system:visualization_3_palette': '#FFFFFF,#FFFF71,#DEFF00,#9EFF00,#77B038,#007E55,#005F51,#004B51,#013A7B,#023AAD',
 'visualization_1_max': '0.1',
 'system:visualization_3_name': 'Daily mean u-component of wind at 10m',
 'visualization_1_name': 'Daily sums of total precipitation in m',
 'visualization_0_max': '320.0',
 'system:visualization_2_max': '1150.0',
 'product_tags': ['wind',
  'temperature',
  'dewpoint',
  'surface',
  'pressure',
  'precipitation',
  'climate'],
 'visualization_1_min': '0.0',
 'system:visualization_2_name': 'Daily mean surface pressure',
 'visuali

In [52]:
display(HTML(coll_info['properties']['description']))

In [61]:
coll_info['features'][0]

{'type': 'Image',
 'bands': [{'id': 'mean_2m_air_temperature',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [1440, 721],
   'crs': 'EPSG:4326',
   'crs_transform': [0.25, 0, -180, 0, -0.25, 90]}],
 'version': 1578495024621102,
 'id': 'ECMWF/ERA5/DAILY/20190701',
 'properties': {'system:time_start': 1561939200000,
  'month': 7,
  'year': 2019,
  'system:footprint': {'type': 'LinearRing',
   'coordinates': [[-180, -90],
    [180, -90],
    [180, 90],
    [-180, 90],
    [-180, -90]]},
  'system:time_end': 1562025600000,
  'system:asset_size': 36605377,
  'day': 1,
  'system:index': '20190701'}}

In [62]:
coll_info['features'][0]['properties']

{'system:time_start': 1561939200000,
 'month': 7,
 'year': 2019,
 'system:footprint': {'type': 'LinearRing',
  'coordinates': [[-180, -90],
   [180, -90],
   [180, 90],
   [-180, 90],
   [-180, -90]]},
 'system:time_end': 1562025600000,
 'system:asset_size': 36605377,
 'day': 1,
 'system:index': '20190701'}

In [48]:
sorted_prop_list = ['year', 'month', 'day', 'system:asset_size']
extract_img_collection_metadata(era5_2mt, sorted_prop_list).loc[:, sorted_prop_list].head(4)

,year,month,day,system:asset_size
0,2019,7,1,36605377
1,2019,7,2,36434216
2,2019,7,3,36553779
3,2019,7,4,36553498
4,2019,7,5,36549748
5,2019,7,6,36610593
6,2019,7,7,36522964
7,2019,7,8,36536766
8,2019,7,9,36489999
9,2019,7,10,36627278


In [35]:
extract_img_collection_properties(era5_2mt)

,IMAGE_DATE,NOMINAL_SCALE,day,month,system:asset_size,system:band_names,system:id,system:index,system:time_end,system:time_start,system:version,year
0,2019-07-01,27829.872698,1,7,36.605377 MB,[mean_2m_air_temperature],ECMWF/ERA5/DAILY/20190701,20190701,2019-07-02 00:00:00,2019-07-01 00:00:00,1.578495e+15,2019
1,2019-07-02,27829.872698,2,7,36.434216 MB,[mean_2m_air_temperature],ECMWF/ERA5/DAILY/20190702,20190702,2019-07-03 00:00:00,2019-07-02 00:00:00,1.578495e+15,2019
2,2019-07-03,27829.872698,3,7,36.553779 MB,[mean_2m_air_temperature],ECMWF/ERA5/DAILY/20190703,20190703,2019-07-04 00:00:00,2019-07-03 00:00:00,1.578495e+15,2019
3,2019-07-04,27829.872698,4,7,36.553498 MB,[mean_2m_air_temperature],ECMWF/ERA5/DAILY/20190704,20190704,2019-07-05 00:00:00,2019-07-04 00:00:00,1.578495e+15,2019
4,2019-07-05,27829.872698,5,7,36.549748 MB,[mean_2m_air_temperature],ECMWF/ERA5/DAILY/20190705,20190705,2019-07-06 00:00:00,2019-07-05 00:00:00,1.578495e+15,2019
5,2019-07-06,27829.872698,6,7,36.610593 MB,[mean_2m_air_temperature],ECMWF/ERA5/DAILY/20190706,20190706,2019-07-07 00:00:00,2019-07-06 00:00:00,1.578495e+15,2019
6,2019-07-07,27829.872698,7,7,36.522964 MB,[mean_2m_air_temperature],ECMWF/ERA5/DAILY/20190707,20190707,2019-07-08 00:00:00,2019-07-07 00:00:00,1.578495e+15,2019
7,2019-07-08,27829.872698,8,7,36.536766 MB,[mean_2m_air_temperature],ECMWF/ERA5/DAILY/20190708,20190708,2019-07-09 00:00:00,2019-07-08 00:00:00,1.578495e+15,2019
8,2019-07-09,27829.872698,9,7,36.489999 MB,[mean_2m_air_temperature],ECMWF/ERA5/DAILY/20190709,20190709,2019-07-10 00:00:00,2019-07-09 00:00:00,1.578495e+15,2019
9,2019-07-10,27829.872698,10,7,36.627278 MB,[mean_2m_air_temperature],ECMWF/ERA5/DAILY/20190710,20190710,2019-07-11 00:00:00,2019-07-10 00:00:00,1.578495e+15,2019


In [65]:
# color gradient for Celsius
celsius_color_grad = {
  'min': -40,
  'max': 35,
  'palette': ['blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}

def func_img_viz(img):
    return img.subtract(273.5).visualize(**celsius_color_grad)

In [66]:
viz_img_coll = era5_2mt.map(func_img_viz)

In [70]:
single_img = extract_single_img(viz_img_coll)

In [78]:
Map = geemap.Map()
Map.addLayer(single_img)
Map.setCenter(0, 25, 2)
Map